# Building a chatbot for basic Financial Services

Pseudo code chatbot

1.	Input box to insert your name Hi … please log in to get in contact with our chatbot
2.	Once name is inserted in the input box the connection is made with the dummy database and country + payment methods (one hot encoding) are available to be accessed
3.	Chatbot starts the conversation with Hi (Name inputted), I am your Financial Assistant, how can I be of support today?
4.	User asks how to pay invoice (wordnet to be used for synonyms of invoice and pay and other similar words). Chatbot retrieves information from the dummy database and retrieves available payment options. Chatbot asks which of the payment methods is wanted by the customer.
5.	User types one of the payment methods – here we need to use potential spelling errors to still be able to understand the intent
6.	User asks where can the open invoice be found, in order to be paid? Chatbot asks invoice number, based on invoice number directs to the correct page (create a format of invoice that is accepted). If invoice number is wrong, display a message to prompt for correct invoice number to be mentioned
7.	Users asks for a copy of the invoice. Same steps as point 6, once invoice number mentioned retrieve a pdf of the invoice or send the link to where the invoice can be found
8.	Users says my invoice is wrong, I need changes made before I can pay (from wordnet get all the combinations of wrong/not correct/incorrect that could be said about invoice being wrong). Chatbot answers I am sorry you consider your invoice to be incorrect. Can you let me know which of these areas is incorrect (show a dropdown menu with Wrong amount/Wrong Billing Details/Wrong Due Date). Then depending on the choice give an answer with a template that needs to be filled in so that the request can be processed 
9.	User asks how they can change their payment method. Chatbot brings the available payment methods from the dummy database and asks for which payment method is wanted. Based on the choice the instructions are displayed
10.	User asks how to change the billing details. Chatbot sends link to place where details can be changed
11.	After each question ask Was the answer helpful? If yes chatbot asks "Is there additional topics you need help with?"  if yes continue with the loop. If no chatbot says May I recommend our recommended payment method Automated payment? You can find more details about the program here. Would you like me to show you how to set it up? If user inputs Yes, Chatbot answers Here is how you can set it up. If user inputs No, then chatbot replies with Thank you for reaching out to us, have a great day!
12.	If No after Was the answer helpful? Chatbot says Sorry to hear that, let me put you in contact with an agent that can help you further


In [1]:
#pip install --upgrade tensorflow

In [2]:
import numpy as np
import pandas as pd
import tkinter as tk
import json 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Import a dummy database which I created to simulate the user accessing the chatbot from whithin a portal where they would need to log in first. In the database there is user Id stored, as well as country and payment methods available for that particular country

In [3]:
users=pd.read_excel('dummy_database1.xlsx')

In [4]:
users.head()

,Country,ID,Female,Credit Card,Bank Transfer,Automated payment
0,Albania,1,Amelia,0,1,1
1,Algeria,2,Fatima,0,1,1
2,Andorra,3,Laia,0,1,1
3,Armenia,4,Nareh,0,1,1
4,Aruba,5,Alysha,0,1,1


Ask for user input to insert the ID of the account - this mimics the log in of the user which would be done automatically through the API in real world usage

In [5]:
user_input = str(input("Hello! Please provide your User ID to get in contact with our chatbot assistant"))

Hello! Please provide your User ID to get in contact with our chatbot assistant14


Boolean to check whether the input value of the ID exists in the database

In [6]:
female_user=users.ID.isin([user_input]).any()

Based on ID retrieve the user name

In [7]:
users_new = users.query("ID == @user_input")
print(users_new["Female"].iloc[0])

Viktoria 


Prompt greeting including the user name. If ID value is not found in the database, message is prompted to insert correct data

In [8]:
#users = cursor.fetchall()
#for user in user_input:
    #print(user)
if female_user == True:
        print("Hello" + " " + (users_new["Female"].iloc[0]) + "," + " " + "I am Fin, your Finance assistant chatbot, how can I help you today?")
else:
        print(str(input("This is not an accepted user ID, please insert the correct user ID and try again")))

Hello Viktoria , I am Fin, your Finance assistant chatbot, how can I help you today?


In [9]:
#use the print statement to understand why it was looping 2 times. 

In [10]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [11]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [12]:
vocab_size = 50000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            800000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 13)                221       
Total params: 800,765
Trainable params: 800,765
Non-trainable params: 0
_________________________________________________________________


In [14]:
epochs = 300
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/300
4/4 [==============================] - 0s 9ms/step - loss: 2.5650 - accuracy: 0.0727
Epoch 2/300
4/4 [==============================] - 0s 14ms/step - loss: 2.5634 - accuracy: 0.1091
Epoch 3/300
4/4 [==============================] - 0s 13ms/step - loss: 2.5623 - accuracy: 0.0909
Epoch 4/300
4/4 [==============================] - 0s 13ms/step - loss: 2.5614 - accuracy: 0.1091
Epoch 5/300
4/4 [==============================] - 0s 11ms/step - loss: 2.5602 - accuracy: 0.1182
Epoch 6/300
4/4 [==============================] - 0s 13ms/step - loss: 2.5589 - accuracy: 0.1273
Epoch 7/300
4/4 [==============================] - 0s 12ms/step - loss: 2.5576 - accuracy: 0.1000
Epoch 8/300
4/4 [==============================] - 0s 16ms/step - loss: 2.5566 - accuracy: 0.1182
Epoch 9/300
4/4 [==============================] - 0s 18ms/step - loss: 2.5550 - accuracy: 0.1545
Epoch 10/300
4/4 [==============================] - 0s 14ms/step - loss: 2.5538 - accuracy: 0.1545
Epoch 11/300
4/4 [==

4/4 [==============================] - 0s 13ms/step - loss: 1.3333 - accuracy: 0.6000
Epoch 84/300
4/4 [==============================] - 0s 12ms/step - loss: 1.3131 - accuracy: 0.6455
Epoch 85/300
4/4 [==============================] - 0s 13ms/step - loss: 1.2932 - accuracy: 0.6455
Epoch 86/300
4/4 [==============================] - 0s 11ms/step - loss: 1.2738 - accuracy: 0.6545
Epoch 87/300
4/4 [==============================] - 0s 11ms/step - loss: 1.2551 - accuracy: 0.6545
Epoch 88/300
4/4 [==============================] - 0s 11ms/step - loss: 1.2362 - accuracy: 0.6727
Epoch 89/300
4/4 [==============================] - 0s 11ms/step - loss: 1.2187 - accuracy: 0.6818
Epoch 90/300
4/4 [==============================] - 0s 12ms/step - loss: 1.2004 - accuracy: 0.6818
Epoch 91/300
4/4 [==============================] - 0s 12ms/step - loss: 1.1842 - accuracy: 0.6636
Epoch 92/300
4/4 [==============================] - 0s 12ms/step - loss: 1.1653 - accuracy: 0.6818
Epoch 93/300
4/4 [=====

4/4 [==============================] - 0s 14ms/step - loss: 0.3264 - accuracy: 0.9636
Epoch 166/300
4/4 [==============================] - 0s 11ms/step - loss: 0.3211 - accuracy: 0.9636
Epoch 167/300
4/4 [==============================] - 0s 11ms/step - loss: 0.3135 - accuracy: 0.9727
Epoch 168/300
4/4 [==============================] - 0s 15ms/step - loss: 0.3065 - accuracy: 0.9727
Epoch 169/300
4/4 [==============================] - 0s 17ms/step - loss: 0.3008 - accuracy: 0.9727
Epoch 170/300
4/4 [==============================] - 0s 16ms/step - loss: 0.2950 - accuracy: 0.9727
Epoch 171/300
4/4 [==============================] - 0s 18ms/step - loss: 0.2883 - accuracy: 0.9727
Epoch 172/300
4/4 [==============================] - 0s 17ms/step - loss: 0.2832 - accuracy: 0.9727
Epoch 173/300
4/4 [==============================] - 0s 21ms/step - loss: 0.2779 - accuracy: 0.9727
Epoch 174/300
4/4 [==============================] - 0s 18ms/step - loss: 0.2716 - accuracy: 0.9727
Epoch 175/300


4/4 [==============================] - 0s 12ms/step - loss: 0.0692 - accuracy: 1.0000
Epoch 248/300
4/4 [==============================] - 0s 17ms/step - loss: 0.0680 - accuracy: 1.0000
Epoch 249/300
4/4 [==============================] - 0s 17ms/step - loss: 0.0669 - accuracy: 1.0000
Epoch 250/300
4/4 [==============================] - 0s 10ms/step - loss: 0.0658 - accuracy: 1.0000
Epoch 251/300
4/4 [==============================] - 0s 10ms/step - loss: 0.0647 - accuracy: 1.0000
Epoch 252/300
4/4 [==============================] - 0s 19ms/step - loss: 0.0639 - accuracy: 1.0000
Epoch 253/300
4/4 [==============================] - 0s 16ms/step - loss: 0.0628 - accuracy: 1.0000
Epoch 254/300
4/4 [==============================] - 0s 19ms/step - loss: 0.0619 - accuracy: 1.0000
Epoch 255/300
4/4 [==============================] - 0s 16ms/step - loss: 0.0608 - accuracy: 1.0000
Epoch 256/300
4/4 [==============================] - 0s 20ms/step - loss: 0.0599 - accuracy: 1.0000
Epoch 257/300


In [15]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: chat_model\assets


In [16]:

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)

def user_inp():
    user_input = str(input("Hello! Please provide your User ID to get in contact with our chatbot assistant"))
    female_user=users.ID.isin([user_input]).any()
    users_new = users.query("ID == @user_input")
    if female_user == True:
        print("Hello" + " " + (users_new["Female"].iloc[0]) + "," + " " + "I am Fin, your Finance assistant chatbot, how can I help you today?")
    else:
        print(str(input("This is not an accepted user ID, please insert the correct user ID and try again")))
#user_inp()  - created the function but do not know how to link it to the below code to make it work   

def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
#create another function based on the tag of payment method to retrieve info from the dummy database

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        #this is the part that needs to be modified to add the name recognition from connection with dummy database
        inp = input()
        #inp = user_inp()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "Fin the ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        #print(Fore.GREEN + "Fin the Chatbot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hello
Fin the ChatBot: Hello, I am Fin, your Finance assistant chatbot, how can I help you today?
User: i want to change my payment method
Fin the ChatBot: Based on your profile the available payment methods are ... Which of these would you like to select?
User: my invoice is wrong
Fin the ChatBot: I am sorry you consider your invoice to be incorrect. Can you let me know which of these areas is incorrect:Wrong amount/Wrong Billing Details/Wrong Due Date)
User: wrong billing
Fin the ChatBot: I see, your billing details on the invoice are not correct. Please mention the invoice number, correct billing details and I will open a case for our Finance Department to come back to you in the shortest time)
User: human agent
Fin the ChatBot: Sorry to hear I cannot help you, let me connect you with a human agent?
User: quit


In [17]:
user_input = str(input("Hello! Please provide your User ID to get in contact with our chatbot assistant"))
female_user=users.ID.isin([user_input]).any()
users_new = users.query("ID == @user_input")
#print(users_new["Female"].iloc[0])
if female_user == True:
        print("Hello" + " " + (users_new["Female"].iloc[0]) + "," + " " + "I am Fin, your Finance assistant chatbot, how can I help you today?")
else:
        print(str(input("This is not an accepted user ID, please insert the correct user ID and try again")))

Hello! Please provide your User ID to get in contact with our chatbot assistant56
Hello Emilija, I am Fin, your Finance assistant chatbot, how can I help you today?


https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

# Problems I need to fix:


1.	How to connect the initial code that recognizes user and greets to the chatbot ML created from intents file? check the comment in the last line of ML code
2.	How to only display 1 time the greeting message (instead of 2 times as it does now)? fixed
3.	How to insert possibility to recognize even wrongly typed words?
4.	How can I make the connection between the user input and the dummy database where I have payment methods mentioned in binary for each user?
5.	How can I connect wordnet with the intents file so that synonyms used can still be recognized even though different phrases than the intents file are used?
6.	How to create the feedback loop (point 5 in the pseudo code) when the user types back the payment method that he wants to use? Additional intents to be created for that?
7.	How to create a feedback loop at the end to ask Was the answer helpful? And Is there an additional topic you need help with?


 point 3 - wrongly typed words - how to recognize that - library? https://towardsdatascience.com/textblob-spelling-correction-46321fc7f8b8
 https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/

point 4 - csv file , once the question occurs, for loop through the list based on the user ID, retrieve the payment methods

point 5 https://www.nltk.org/howto/wordnet.html 

add another intent to say Please connect me with a human agent - done